In [1]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd
import os
import tqdm

In [2]:
class IDAO_validator_2d_model:
    """
    Валидация для 2d модели
    """
    
    def __init__(self, parts_amount, max_steps_amount):
        """
        parts_amount - 
        """
        
        self.parts_amount = parts_amount
        self.max_steps_amount = max_steps_amount
        
    def get_n_splits(self):
        return self.n_splits
    
    def split(self, X):
        """
        На выходе будет 2 двумерных numpy массив размера (sat_id, 2). Один для трейна, другой для теста.
        Каждая строка соответствует своему sat_id
        Значения 0 столбца - индекс начала трейна (теста) для данного sat_id
        Значения 1 столбца - индекс конца трейна (теста) для данного sat_id
        """
        
        for step in np.arange(1, self.max_steps_amount + 1):
            offset = 0
            train_idxs = np.zeros((X['sat_id'].nunique(), 2))
            test_idxs = np.zeros((X['sat_id'].nunique(), 2))
            for sat_id in X['sat_id'].unique():
                if sat_id != 0:
                    offset += len(X[X['sat_id'] == (sat_id - 1)])
                train_idxs[sat_id, 0] = offset
                train_idxs[sat_id, 1] = offset + int(step / self.parts_amount * X[X['sat_id'] == sat_id].shape[0])
                
                if (step + 1) / self.parts_amount > 1.0:
                    print('WARNING: Не весь тест момещается -> пропускаем фолд')
                    continue
                
                test_idxs[sat_id, 0] = offset + int(step / self.parts_amount * X[X['sat_id'] == sat_id].shape[0])
                test_idxs[sat_id, 1] = offset + int((step + 1) / self.parts_amount * X[X['sat_id'] == sat_id].shape[0])
                
            yield (train_idxs.astype(int), test_idxs.astype(int))
        
        

In [3]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [4]:
# Загрузка данных
# windows
PATH_TO_DATA = os.path.join('../data')
full_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col='id')
full_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'), index_col='id')

In [5]:
idao_2d_splitter = IDAO_validator_2d_model(5, 5)

In [7]:
import lightgbm

In [8]:
%%time

## колонки, которые не нужны для предсказания
columns_to_drop = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'epoch', 'sat_id']

## целевые колонки
target_columns = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

## номер фолда для вывода логов
n_fold = 0
## предыдущий 'sat_id' - опять же для вывода логов - чтобы отслеживать смену объектов (sat_id)
prev_sat_id = 0

## список из значений smape (на тесте) для каждого из sat_id
smape_per_sat_id = []

for trn_idxs, tst_idxs in idao_2d_splitter.split(full_train):
    
    ## для каждого sat_id обновляем n_fold (для логов)
    for sat_id in full_train['sat_id'].unique():
        ## создаем массив для хранения предсказаний для теста, чтобы посчитать smape
        preds = np.zeros((tst_idxs[sat_id, 1] - tst_idxs[sat_id, 0], len(target_columns)))
        ## массив для хранения реальных значений целевых переменных для теста
        y_true = np.zeros((tst_idxs[sat_id, 1] - tst_idxs[sat_id, 0], len(target_columns)))
        ## цикл по целевым колонкам
        for i, col in enumerate(target_columns):

            lgbm = lightgbm.LGBMRegressor(max_depth=5, learning_rate=0.2, n_estimators=1000, reg_alpha=5e-6,
                                          colsample_bytree = 0.85,
                                          reg_lambda=5e-4,
                                          random_state=13)
            X_train = full_train.iloc[trn_idxs[sat_id, 0]:trn_idxs[sat_id, 1]].drop(columns=columns_to_drop).values
            y_train = full_train.iloc[trn_idxs[sat_id, 0]:trn_idxs[sat_id, 1]][col].values

            X_test = full_train.iloc[tst_idxs[sat_id, 0]:tst_idxs[sat_id, 1]].drop(columns=columns_to_drop).values
            y_test = full_train.iloc[tst_idxs[sat_id, 0]:tst_idxs[sat_id, 1]][col].values
            if X_test.shape[0] != 0:
                lgbm.fit(X_train, y_train)

                preds[:, i] += lgbm.predict(X_test)
                y_true[:, i] = y_test

            ## выводим метрику для каждой целевой колонки отдельно
            print(f'sat_id: {sat_id}, fold: {n_fold}, target_col: {col},\
                  smape: {1 - smape(preds[:, i], y_test):.5f}')
            
        ## выводим метрику для всех целевых колонок сразу (для одного sat_id)
        print('_' * 15)
        print(f'sat_id: {sat_id}, fold: {n_fold}, target_col: ALL_COLUMNS,\
                  smape: {1 - smape(preds, y_true):.5f}')

        smape_per_sat_id.append(smape(preds, y_true))
        print('_' * 15)
    n_fold += 1

sat_id: 0, fold: 0, target_col: x,                  smape: 0.85068
sat_id: 0, fold: 0, target_col: y,                  smape: 0.85312
sat_id: 0, fold: 0, target_col: z,                  smape: 0.84829
sat_id: 0, fold: 0, target_col: Vx,                  smape: 0.85590
sat_id: 0, fold: 0, target_col: Vy,                  smape: 0.82734
sat_id: 0, fold: 0, target_col: Vz,                  smape: 0.85745
_______________
sat_id: 0, fold: 0, target_col: ALL_COLUMNS,                  smape: 0.84880
_______________
sat_id: 1, fold: 0, target_col: x,                  smape: 0.93968
sat_id: 1, fold: 0, target_col: y,                  smape: 0.94248
sat_id: 1, fold: 0, target_col: z,                  smape: 0.93829
sat_id: 1, fold: 0, target_col: Vx,                  smape: 0.94498
sat_id: 1, fold: 0, target_col: Vy,                  smape: 0.93642
sat_id: 1, fold: 0, target_col: Vz,                  smape: 0.93382
_______________
sat_id: 1, fold: 0, target_col: ALL_COLUMNS,                  sma

sat_id: 17, fold: 0, target_col: x,                  smape: 0.86117
sat_id: 17, fold: 0, target_col: y,                  smape: 0.84061
sat_id: 17, fold: 0, target_col: z,                  smape: 0.85579
sat_id: 17, fold: 0, target_col: Vx,                  smape: 0.83442
sat_id: 17, fold: 0, target_col: Vy,                  smape: 0.84266
sat_id: 17, fold: 0, target_col: Vz,                  smape: 0.83076
_______________
sat_id: 17, fold: 0, target_col: ALL_COLUMNS,                  smape: 0.84424
_______________
sat_id: 18, fold: 0, target_col: x,                  smape: 0.81301
sat_id: 18, fold: 0, target_col: y,                  smape: 0.81696
sat_id: 18, fold: 0, target_col: z,                  smape: 0.80408
sat_id: 18, fold: 0, target_col: Vx,                  smape: 0.79463
sat_id: 18, fold: 0, target_col: Vy,                  smape: 0.80616
sat_id: 18, fold: 0, target_col: Vz,                  smape: 0.80267
_______________
sat_id: 18, fold: 0, target_col: ALL_COLUMNS,       

sat_id: 33, fold: 0, target_col: x,                  smape: 0.80031
sat_id: 33, fold: 0, target_col: y,                  smape: 0.80489
sat_id: 33, fold: 0, target_col: z,                  smape: 0.80740
sat_id: 33, fold: 0, target_col: Vx,                  smape: 0.79614
sat_id: 33, fold: 0, target_col: Vy,                  smape: 0.78265
sat_id: 33, fold: 0, target_col: Vz,                  smape: 0.80553
_______________
sat_id: 33, fold: 0, target_col: ALL_COLUMNS,                  smape: 0.79949
_______________
sat_id: 34, fold: 0, target_col: x,                  smape: 0.94755
sat_id: 34, fold: 0, target_col: y,                  smape: 0.99179
sat_id: 34, fold: 0, target_col: z,                  smape: 0.99369
sat_id: 34, fold: 0, target_col: Vx,                  smape: 0.99409
sat_id: 34, fold: 0, target_col: Vy,                  smape: 0.96772
sat_id: 34, fold: 0, target_col: Vz,                  smape: 0.99259
_______________
sat_id: 34, fold: 0, target_col: ALL_COLUMNS,       

sat_id: 49, fold: 0, target_col: x,                  smape: 0.93410
sat_id: 49, fold: 0, target_col: y,                  smape: 0.95430
sat_id: 49, fold: 0, target_col: z,                  smape: 0.96656
sat_id: 49, fold: 0, target_col: Vx,                  smape: 0.96258
sat_id: 49, fold: 0, target_col: Vy,                  smape: 0.93502
sat_id: 49, fold: 0, target_col: Vz,                  smape: 0.94863
_______________
sat_id: 49, fold: 0, target_col: ALL_COLUMNS,                  smape: 0.95020
_______________
sat_id: 50, fold: 0, target_col: x,                  smape: 0.83408
sat_id: 50, fold: 0, target_col: y,                  smape: 0.81917
sat_id: 50, fold: 0, target_col: z,                  smape: 0.83072
sat_id: 50, fold: 0, target_col: Vx,                  smape: 0.81779
sat_id: 50, fold: 0, target_col: Vy,                  smape: 0.78015
sat_id: 50, fold: 0, target_col: Vz,                  smape: 0.80520
_______________
sat_id: 50, fold: 0, target_col: ALL_COLUMNS,       

sat_id: 65, fold: 0, target_col: x,                  smape: 0.82540
sat_id: 65, fold: 0, target_col: y,                  smape: 0.80668
sat_id: 65, fold: 0, target_col: z,                  smape: 0.82837
sat_id: 65, fold: 0, target_col: Vx,                  smape: 0.79845
sat_id: 65, fold: 0, target_col: Vy,                  smape: 0.81929
sat_id: 65, fold: 0, target_col: Vz,                  smape: 0.80267
_______________
sat_id: 65, fold: 0, target_col: ALL_COLUMNS,                  smape: 0.81348
_______________
sat_id: 66, fold: 0, target_col: x,                  smape: 0.31185
sat_id: 66, fold: 0, target_col: y,                  smape: 0.60926
sat_id: 66, fold: 0, target_col: z,                  smape: 0.64316
sat_id: 66, fold: 0, target_col: Vx,                  smape: 0.23407
sat_id: 66, fold: 0, target_col: Vy,                  smape: 0.14055
sat_id: 66, fold: 0, target_col: Vz,                  smape: 0.19470
_______________
sat_id: 66, fold: 0, target_col: ALL_COLUMNS,       

...

sat_id: 538, fold: 3, target_col: Vz,                  smape: 0.90681
_______________
sat_id: 538, fold: 3, target_col: ALL_COLUMNS,                  smape: 0.91591
_______________
sat_id: 539, fold: 3, target_col: x,                  smape: 0.91794
sat_id: 539, fold: 3, target_col: y,                  smape: 0.93719
sat_id: 539, fold: 3, target_col: z,                  smape: 0.97911
sat_id: 539, fold: 3, target_col: Vx,                  smape: 0.96639
sat_id: 539, fold: 3, target_col: Vy,                  smape: 0.92278
sat_id: 539, fold: 3, target_col: Vz,                  smape: 0.95792
_______________
sat_id: 539, fold: 3, target_col: ALL_COLUMNS,                  smape: 0.94689
_______________
sat_id: 540, fold: 3, target_col: x,                  smape: 0.99257
sat_id: 540, fold: 3, target_col: y,                  smape: 0.99481
sat_id: 540, fold: 3, target_col: z,                  smape: 0.99658
sat_id: 540, fold: 3, target_col: Vx,                  smape: 0.99621
sat_id: 540, f

sat_id: 554, fold: 3, target_col: Vx,                  smape: 0.94875
sat_id: 554, fold: 3, target_col: Vy,                  smape: 0.96752
sat_id: 554, fold: 3, target_col: Vz,                  smape: 0.97229
_______________
sat_id: 554, fold: 3, target_col: ALL_COLUMNS,                  smape: 0.96357
_______________
sat_id: 555, fold: 3, target_col: x,                  smape: 0.86003
sat_id: 555, fold: 3, target_col: y,                  smape: 0.89457
sat_id: 555, fold: 3, target_col: z,                  smape: 0.82591
sat_id: 555, fold: 3, target_col: Vx,                  smape: 0.85513
sat_id: 555, fold: 3, target_col: Vy,                  smape: 0.87388
sat_id: 555, fold: 3, target_col: Vz,                  smape: 0.82896
_______________
sat_id: 555, fold: 3, target_col: ALL_COLUMNS,                  smape: 0.85641
_______________
sat_id: 556, fold: 3, target_col: x,                  smape: 0.83564
sat_id: 556, fold: 3, target_col: y,                  smape: 0.75558
sat_id: 556, 

sat_id: 570, fold: 3, target_col: x,                  smape: 0.97037
sat_id: 570, fold: 3, target_col: y,                  smape: 0.94704
sat_id: 570, fold: 3, target_col: z,                  smape: 0.95372
sat_id: 570, fold: 3, target_col: Vx,                  smape: 0.95914
sat_id: 570, fold: 3, target_col: Vy,                  smape: 0.90555
sat_id: 570, fold: 3, target_col: Vz,                  smape: 0.95200
_______________
sat_id: 570, fold: 3, target_col: ALL_COLUMNS,                  smape: 0.94797
_______________
sat_id: 571, fold: 3, target_col: x,                  smape: 0.90671
sat_id: 571, fold: 3, target_col: y,                  smape: 0.95875
sat_id: 571, fold: 3, target_col: z,                  smape: 0.91111
sat_id: 571, fold: 3, target_col: Vx,                  smape: 0.90212
sat_id: 571, fold: 3, target_col: Vy,                  smape: 0.92977
sat_id: 571, fold: 3, target_col: Vz,                  smape: 0.94454
_______________
sat_id: 571, fold: 3, target_col: ALL_C

sat_id: 585, fold: 3, target_col: Vy,                  smape: 0.85157
sat_id: 585, fold: 3, target_col: Vz,                  smape: 0.87454
_______________
sat_id: 585, fold: 3, target_col: ALL_COLUMNS,                  smape: 0.85839
_______________
sat_id: 586, fold: 3, target_col: x,                  smape: 0.72812
sat_id: 586, fold: 3, target_col: y,                  smape: 0.72580
sat_id: 586, fold: 3, target_col: z,                  smape: 0.69932
sat_id: 586, fold: 3, target_col: Vx,                  smape: 0.71843
sat_id: 586, fold: 3, target_col: Vy,                  smape: 0.67109
sat_id: 586, fold: 3, target_col: Vz,                  smape: 0.64751
_______________
sat_id: 586, fold: 3, target_col: ALL_COLUMNS,                  smape: 0.69838
_______________
sat_id: 587, fold: 3, target_col: x,                  smape: 0.41235
sat_id: 587, fold: 3, target_col: y,                  smape: 0.34502
sat_id: 587, fold: 3, target_col: z,                  smape: 0.33898
sat_id: 587, f

ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required.

In [12]:
print(1 - np.mean(smape_per_sat_id))

0.8961579202144785
